# 2016 Canadian Census Analysis

In [6]:
import pandas as pd

## CMA Population Changes from 2011 to 2016

In [9]:
raw = pd.read_csv("../raw/RAW 2016 STATSCAN CENSUS DATA.csv", low_memory=False)

In [10]:
population_raw = raw[raw["DIM: Profile of Census Metropolitan Areas/Census Agglomerations (2247)"].isin(["Population, 2016", "Population, 2011"])]

population_pivot = pd.pivot(population_raw, index=["GEO_CODE (POR)", "GEO_NAME"], columns="DIM: Profile of Census Metropolitan Areas/Census Agglomerations (2247)", values="Dim: Sex (3): Member ID: [1]: Total - Sex")
population_pivot["pct_change"] = (population_pivot["Population, 2016"].astype(int) - population_pivot["Population, 2011"].astype(int)) / population_pivot["Population, 2011"].astype(int)

In [12]:
cmas = population_raw[["ALT_GEO_CODE", "GEO_NAME"]].drop_duplicates(subset="ALT_GEO_CODE")
code = (cmas["ALT_GEO_CODE"]
        .astype(str)
        .str[-3:].astype(int))

0           1
2247        5
4494       10
6741       11
8988       15
         ... 
348285    970
350532    975
352779    977
355026    990
357273    995
Name: ALT_GEO_CODE, Length: 160, dtype: int32

## Change from 2011 to 2016

### Census subdivisions

In [ ]:
raw_csd = pd.read_csv("../raw/RAW 2016 STATSCAN CENSUS CSD.csv", low_memory=False)
csd = raw_csd[raw_csd["GEO_LEVEL"] == 3]

csd = (csd[csd["DIM: Profile of Census Divisions/Census Subdivisions (2247)"] == "Population percentage change, 2011 to 2016"]
       .pivot(index=["GEO_CODE (POR)", "GEO_NAME"], columns="DIM: Profile of Census Divisions/Census Subdivisions (2247)", values="Dim: Sex (3): Member ID: [1]: Total - Sex")
       .reset_index()
       )

### Census tracts

In [ ]:
tracts_raw = pd.read_csv("../raw/RAW 2016 STATSCAN CENSUS TRACTS.csv", low_memory=False)

tracts_population = tracts_raw[tracts_raw["DIM: Profile of Census Tracts (2247)"] == "Population percentage change, 2011 to 2016"]
tracts_population = tracts_population[tracts_population["GEO_LEVEL"] == 2]

tracts_pivot = tracts_population.pivot(columns="DIM: Profile of Census Tracts (2247)", index=["GEO_CODE (POR)", "GEO_NAME"], values="Dim: Sex (3): Member ID: [1]: Total - Sex").reset_index().set_index("GEO_CODE (POR)")
tracts_pivot["CMA_code"] = tracts_pivot.index.astype(str).str[0:3].astype(int)
tracts_pivot = tracts_pivot.merge(cmas, how="left", left_on="CMA_code", right_on="ALT_GEO_CODE")
tracts_pivot = tracts_pivot[["GEO_NAME_x", "GEO_NAME_y", "Population percentage change, 2011 to 2016"]]